### Problema: Predicción de precios de viviendas.

El dataset `California Housing` es un corpus que contiene información sobre el valor medio del precio de las viviendas unifamiliares en los condados de California. 

Contiene 20.640 instancias, cada una con 8 características que describen diversos aspectos de la vivienda y su ubicación.  

**Objetivo**: Predecir el precio medio de las viviendas en miles de dólares.

**Cargar el dataset** y comprobar las instancias y características.

In [6]:
from sklearn.datasets import fetch_california_housing
cali_housing = fetch_california_housing()
cali_housing.data.shape

Crear un DataFrame de Pandas a partir de los datos para visualizar las características.

`cali_housing.data` es una matriz NumPy que contiene los valores de las características, y `cali_housing.feature_names` es una lista de los nombres de las características. 

`cali_housing.target` es una matriz NumPy que contiene los valores objetivo (valor medio de las viviendas).

In [2]:
import pandas as pd
df = pd.DataFrame(cali_housing.data, columns=cali_housing.feature_names)
df['MedHouseValue'] = cali_housing.target

In [3]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseValue
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


**¿Clasificación o Regresión?**

Dividir la matriz de características y el vector de clases en los conjuntos de entrenamiento y prueba. Utilicemos el 70% de los datos para entrenar.

Se puede especificar un `random_state` que causa que la división sea determinista.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(cali_housing.data, cali_housing.target, test_size=0.3, random_state=42)

In [5]:
print(X_train.shape)
print(X_test.shape)

Para entrenar el clasificador con los datos extraídos en sklearn, se utiliza la función `fit`. Recibe la matriz y el vector de clase de entrenamiento. 

In [1]:
from sklearn.linear_model import LinearRegression

lr=LinearRegression()
lr.fit(X_train, y_train)

Para evaluar el clasificador se pueden utilizar diversas métricas de sklearn que deben depender del problema y el modelo utilizado.

In [3]:
from sklearn.metrics import mean_squared_error, r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = lr.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Error cuadrático medio:', mse)
print('Puntuación R2:', r2)

### ¿Cómo se puede representar datos complejos, no naturalmente tabulares, para realizar tareas de clasificación o regresión sobre ellos?
Por ejemplo: 
- Dado un audio de 5 segundos clasificar que tipo de sonido es.
- Dado una imagen determinar si es de día o de noche.
- Dado un texto determinar los sentimientos o emociones en él.


### Problema: Análisis de críticas de cine.

El dataset `txt_sentoken` es un corpus de críticas de películas con etiquetas positivas o negativas. 

Cada elemento del dataset es un objeto `Sentiment` que contiene los siguientes atributos:

- `data`: El texto sin procesar de la crítica.
- `target`: La etiqueta de sentimiento (0 para negativo y 1 para positivo).
  
El dataset se organiza en dos carpetas:

- `pos`: Contiene archivos que contienen críticas positivas.
- `neg`: Contiene archivos que contienen críticas negativas.

Cada archivo en estas carpetas representa una crítica individual.

**Objetivo**: Aprender a determinar si una crítica es positiva o negativa basándose en su texto.

**Cargar el dataset**: Primeramente cargar los archivos. Luego hacer una lista con los textos extraídos.

In [4]:
from pathlib import Path

# directorio con los archivos de críticas positivas
path_p = Path("data/txt_sentoken/pos") 
# directorio con los archivos de críticas negativas
path_n = Path("data/txt_sentoken/neg")

# listas con el path a cada archivo de los directorios pos y neg
ds_p = list(path_p.iterdir()) 
ds_n = list(path_n.iterdir())

In [5]:
def convert_file_to_text(file_path: Path) -> str:
    with file_path.open('r', encoding='utf-8') as file:
        return file.read()

In [6]:
texts_p = []    # Lista de críticas positivas
texts_n = []    # Lista de críticas negativas

for file in ds_p:
    texts_p.append(convert_file_to_text(file))

for file in ds_n:
    texts_n.append(convert_file_to_text(file))

Comprobación de que estén todas las críticas

In [4]:
print(len(texts_p), len(texts_n))

**¿Cómo convertimos una cadena de texto en una matriz de características?**

Se puede seleccionar varios métodos de vectorizar texto con sklearn. Una de las maneras es `CountVectorizer`, que convierte una colección de documentos en una matriz con recuentos de tokens.

Para crear la matriz de características con `CountVectorizer`, se usa su método `fit_transform`. Cada fila de la matriz representa un documento y cada columna representa una palabra única en el corpus. Representa el número de veces que ocurre un término para cada documento.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts_p + texts_n)
Xa = X.toarray()

La matriz de características tiene dos dimensiones, la primera representa la cantidad de instancias (ejemplos) y la segunda la cantidad de características. Busquemos las dimensiones de la matriz:

In [7]:
# matriz de características
X.shape

Para cada documento, ¿cuál es el por ciento de términos cuya ocurrencia es distinto de cero? Es decir, ¿cuál es el por ciento de los valores en la matriz que son distintos de cero? 

In [8]:
X.nnz * 100.0 / (X.shape[0] * X.shape[1])

En clasificación se necesitan los valores correctos para cada instancia del dataset como parte del aprendizaje. 

Entonces para cada crítica debemos indicar a que clase pertenece. La clase positiva (1) representa una crítica positiva y la clase negativa (0) una crítica negativa.

In [12]:
y = [1]*1000 + [0]*1000

#### Elegir un clasificador

El Naive Bayes de sklearn se llama Gaussian Naive Bayes. 
<!-- porque está diseñado (a diferencia del visto en conferencia) para lidiar con características que sean valores continuos. Gaussian Naive Bayes supone que la probabilidad de las características es gaussiana: $$P(x_i ∣ y)=\dfrac{1}{\sqrt{2\pi \sigma^2_y}}exp(−\dfrac{{(x_i-\mu_y)^2}}{2\sigma^2_y})$$ -->

In [13]:
from sklearn.naive_bayes import GaussianNB

Dividir la matriz de características y el vector de clases en los conjuntos de entrenamiento y prueba. Utilicemos el 60% de los datos para entrenar.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [9]:
print(X_train.shape)
print(X_test.shape)

Para entrenar el clasificador con los datos extraídos en sklearn, se utiliza la función `fit`. Recibe la matriz y el vector de clase de entrenamiento. 

In [10]:
clf = GaussianNB()

clf.fit(X_train.toarray(), y_train)

Para evaluar el clasificador se pueden utilizar las métricas de sklearn como `score`, que evalúa la precisión. Para ello se le da un conjunto de datos a evaluar y el correspondiente vector de clases.

In [11]:
accuracy = clf.score(X_test.toarray(), y_test)
print(f"Accuracy: {accuracy}")

**¿Cómo se podrían mejorar los resultados sin cambiar el modelo?**

**Preprocesamiento del texto**: Limpiar y normalizar el texto (eliminar puntuación, convertir a minúsculas, eliminar stop words, etc.).

**Ajustar los hiperparámetros del vectorizador**: Probar diferentes configuraciones de CountVectorizer, como ngram_range, max_features, etc.

**Utilizar técnicas de reducción de dimensionalidad**: Aplicar técnicas como PCA (Principal Component Analysis) para reducir la dimensionalidad de los datos.

**Aumentar el tamaño del dataset**: Si es posible, agregar más datos para mejorar la capacidad del modelo para generalizar.

**Utilizar técnicas de balanceo de clases**: Si las clases están desbalanceadas, utilizar técnicas como sobremuestreo o submuestreo para balancearlas.